<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Usando la API para obtener datos sobre personas desaparecidas del RNPDNO


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/RNPDNO-API.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [1]:
import os
import sys
import requests
import datetime

import pandas as pd
import json


## Calentando motores

Para descargar los datos, vamos a consultar directamente la base de datos pública del [Registro Nacional de Personas Desaparecidas y No Localizadas (RNPDNO)](https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index).

El RNPDNO no tiene una API tal cual, sin embargo, [Pablo Reyes Moctezuma](https://github.com/pablorm296) encontró una manera de extraer la información usando la librería `request` de python. La API que, me imagino, el extrajo a punta de prueba y error la documento en [este archivo en markdown](https://github.com/pablorm296/ScrapperRNPDNO/blob/master/Test/API.md). Un chambón.

Vamos air la usando poco a poco, empecemos por tratar de encontrar en el catálogo los indices de estados, municipios y colonias.

In [2]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

Los identificadores de los estados:

In [3]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']

estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


y ahora los municipios de Sonora

In [4]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_id

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
...,...,...
69,65,TUBUTAMA
70,66,URES
71,67,VILLA HIDALGO
72,68,VILLA PESQUEIRA


y por último los identificadores de las colonias del municipio de Hermosillo

In [5]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_COL
DATA = {"idEstado": "26", "idMunicipio": "30"}

r = main_session.post(TARGET_URL, data = DATA)
col_hmo_id = pd.json_normalize(r.json())
col_hmo_id.columns = ['Valor', 'Municipio']
col_hmo_id

,Valor,Municipio
0,0,--TODAS--
1,347025,22 DE SEPTIEMBRE
2,347004,26 DE OCTUBRE
3,347026,4 DE MARZO
4,347027,4 OLIVOS
...,...,...
631,347358,VISTA DEL LAGO
632,347359,Y
633,347582,ZACATON
634,347384,ZAMORA


## Sociodemográficos totales

`Con este `endpoint` se pueden consultar resúmenes generales de la información que se pide. recuerda de revisar los catálogos.

Hay dos variables cuyos valores posibles son los siguientes:

**idEstatusVictima**:
- "0" PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS
- "2" PERSONAS LOCALIZADAS CON VIDA
- "3" PERSONAS LOCALIZADAS SIN VIDA- "4" PERSONAS DESAPARECIDAS
- "5" PERSONAS NO LOCALIZADAS
- "6" PERSONAS LOCALIZADAS
- "7" PERSONAS DESAPARECIDAS Y NO LOCALIZADAS

**idHipotesisNoLocalizacion**:
- "0" --TODAS--
- "1" ACCIDENTE
- "2" CATÁSTROFE
- "3" NO LOCALIZACIÓN VOLUNTARIA
- "4" NO LOCALIZACIÓN INVOLUNTARIA
- "5" SE DESCONOCE

Veamos como funciona pidiendo información de Sonora y de Hermosillo. Empecemos por Sonora

In [6]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"

DATA = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

resumen_sonora = pd.json_normalize(r.json()).T
resumen_sonora.columns = ['Valor']

resumen_sonora


,Valor
TotalGlobal,"7,153"
TotalDesaparecidos,"4,322"
TotalLocalizados,"2,831"
PorcentajeDesaparecidos,60.42 %
PorcentajeLocalizados,39.58 %
TotalSoloDesaparecidos,"4,277"
TotalSoloNoLocalizados,45
PorcentajeSoloDesaparecidos,98.96 %
PorcentajeSoloNoLocalizados,1.04 %
TotalLocalizadosCV,"2,609"


### Ejercicio

Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

In [7]:
# Se usa el endpoint que ofrece datos sobre los rangos de edad de las personas desaparecidas
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoRango"

# Con este DATA para especificar el Municipio de cajeme
DATA = {
    "titulo": "PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo": "POR COLONIAS - CAJEME",
    "idEstatusVictima": "0",
    "fechaInicio": "",
    "fechaFin": "",
    "idEstado": "26",
    "idMunicipio": "18",
    "mostrarFechaNula": "0",
    "idColonia": "0",
    "idNacionalidad": "0",
    "edadInicio": "",
    "edadFin": "",
    "mostrarEdadNula": "0",
    "idHipotesis": "",
    "idMedioConocimiento": "",
    "idCircunstancia": "",
    "tieneDiscapacidad": "",
    "idTipoDiscapacidad": "0",
    "idEtnia": "0",
    "idLengua": "0",
    "idReligion": "",
    "esMigrante": "",
    "idEstatusMigratorio": "0",
    "esLgbttti": "",
    "esServidorPublico": "",
    "esDefensorDH": "",
    "esPeriodista": "",
    "esSindicalista": "",
    "esONG": "",
    "idHipotesisNoLocalizacion": "0",
    "idDelito": "0"
}
r = main_session.post(TARGET_URL, json = DATA)

# La respuesta de la petición guarda como un diccionario de python para poder extraer la información de ahí
data = r.json()

# El x_axis_categories hace referencia a los rangos de edad
x_axis_categories = data["XAxisCategories"]

# La respuesta tiene una Serie para Hombre, Mujer e Indeterminado, se obtiene la información solamente de la Serie Mujer
for serie in data["Series"]:
    if serie["name"] == "Mujer":
        mujer_data = serie["data"]

# Se guarda la información capturada como un dataframe
mujeres_rangoedad = pd.DataFrame({'Rango de edad':x_axis_categories,'Mujeres Desaparecidas':mujer_data})
mujeres_rangoedad
# Mujeres desaparecidas en el municipio de cajeme por rango de edad.

,Rango de edad,Mujeres Desaparecidas
0,0-4,6
1,5-9,0
2,10-14,29
3,15-19,59
4,20-24,26
5,25-29,17
6,30-34,15
7,35-39,18
8,40-44,3
9,45-49,6


In [8]:
# Se puede extraer el porcentaje de personas desaparecidas en Cajeme que corresponde a mujeres de una forma muy parecida
# Igual para todo esto es importante saber como son las respuesta del archivo json que da como respuesta el endpoint

for serie in data["TableValues"]:
    if serie["text"] == "Mujer":
        porcentaje = serie["porcent"]

print(porcentaje)

33.18%


In [9]:
# Facilmente se puede obtener todo el TableValues como un dataframe

tablevalues = pd.DataFrame(data["TableValues"])
tablevalues

,text,value,porcent
0,Hombre,574,66.82%
1,Mujer,285,33.18%
2,Indeterminado,00,0.00%
3,<strong>Total</strong>,859,100.00%


## Personas desaparecidas por sexo y colonia

El `endpoint` **BarChartSexoColonia** está diseñado para generar gráficas, pero nos permite extraer información, si la sabemos formatear.

Vamos viendo un ejemplo:


In [10]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"25",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
serie_colonias = {'Colonia': res['XAxisCategories']}
series_sexo = {serie['name']: serie['data'] for serie in res['Series']}
datos = serie_colonias | series_sexo
por_colonia = pd.DataFrame(datos).set_index('Colonia')
por_colonia



,Hombre,Mujer,Indeterminado
Colonia,,,
PESQUEIRA,0,1,0
SAHUARAL,1,0,0
RONY CAMACHO,2,0,0
ORTIZ RUBIO,1,0,0
MAYTORENA,1,0,0
LADRILLERA,1,0,0
LIBERTAD,1,0,0
JUÁREZ,0,1,0
VILLA DORADA,2,0,0


### Ejercicio

¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

In [21]:
def tabla_desaparecidos(estado = 0, municipio = 0):
    
    """
    Para obtener un dataframe con la cantidad de personas desaparecidas por estado ejecute la función sin argumentos.

    Si lo que quiere es un data frame con los datos de municipios en un estado utilicé un solo argumento que corresponda
    al id del estado de interes.

    Para obtener los datos de las colonias de un municipio, utilice los dos argumentos de la función, de la forma

    tabla_desaparecidos(estado = 26, municipio = 25) sería el dataframe de los desaparecidos por colonia de Empalme, Sonora.

    tabla_desaparecidos(26) sería el dataframe de los desaparecidos por municipio en Sonora.

    WARNING! Funciona extricmante en este sentido. Escribir, por ejemplo, tabla_desaparecidos(municipio = 30)
    devolverá a los desaparecidos por estado, equivalente a escribir tabla_desaparecidos().
    Necesariamente se ocupa un estado de referencia para obtener la información por colonias.
    
    """
    if estado == 0:
        LEAF = 'BarChartSexoEstados'
    elif municipio == 0:
        LEAF = 'BarChartSexoMunicipio'
    else:
        LEAF = 'BarChartSexoColonia'
    
    TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + LEAF

    DATA = {
    "idEstado":str(estado),
    "idMunicipio":str(municipio),
    "idColonia":"0",
    }

    r = main_session.post(TARGET_URL, json = DATA)
    res = r.json()
    serie_region = {'Región': res['XAxisCategories']}
    series_sexo = {serie['name']: serie['data'] for serie in res['Series']}
    datos = serie_region | series_sexo
    por_region = pd.DataFrame(datos).set_index('Región')
    
    return por_region

In [22]:
tabla_desaparecidos(estado = 26, municipio = 25)

,Hombre,Mujer,Indeterminado
Región,,,
PESQUEIRA,0,1,0
SAHUARAL,1,0,0
RONY CAMACHO,2,0,0
ORTIZ RUBIO,1,0,0
MAYTORENA,1,0,0
LADRILLERA,1,0,0
LIBERTAD,1,0,0
JUÁREZ,0,1,0
VILLA DORADA,2,0,0


## Información por sexo y por año

Tambien se puede encontrar información por sexo y por año utilizando otro `endpoint`: **AreaChartSexoAnio**

Sin mas choro, vamos a ver como se usa, otra vez con el estado de Sonora:

In [23]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

serie_fecha={'Fecha': res['XAxisCategories']} 
series = {serie['name']: serie['data'] for serie in res['Series']}
datos = serie_fecha | series
por_fecha = pd.DataFrame(datos)
por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')

por_fecha.set_index('Fecha')


,Hombre,Mujer,Indeterminado
Fecha,,,
NaN,124,45,5
1974.0,3,0,0
1977.0,1,0,0
1978.0,2,0,0
1980.0,1,0,0
1981.0,7,1,0
1982.0,1,0,0
1989.0,1,0,0
1994.0,2,0,0


### Ejercicio

¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

In [24]:
def tabla_desaparecidos_anio(estado = 0, municipio = 0):
    
    TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

    DATA = {
    "idEstado":str(estado),
    "idMunicipio":str(municipio),
    "idColonia":"0",
    }

    r = main_session.post(TARGET_URL, json = DATA)

    res = r.json()

    serie_fecha={'Fecha': res['XAxisCategories']} 
    series = {serie['name']: serie['data'] for serie in res['Series']}
    datos = serie_fecha | series
    por_fecha = pd.DataFrame(datos)
    por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')

    return por_fecha.set_index('Fecha')

In [27]:
tabla_desaparecidos_anio(estado = 26, municipio=25)

,Hombre,Mujer,Indeterminado
Fecha,,,
NaN,4,2,0
2011.0,2,2,0
2012.0,4,0,0
2013.0,0,1,0
2014.0,2,0,0
2015.0,4,2,0
2016.0,7,5,0
2017.0,11,5,0
2018.0,19,9,0


### Ejercicio

Extrae alguna información del conjunto de tados que pienses que es relevante, y explica porqué.

## Practicando a ser investigador de APIs

Ahora te pido que revises si puedes encontrar otros endpoints para recuperar mas información de las bases que no se encuentran liberadas.

Puede ser en la misma página, o en blogs o revisando código.

Agrega en esta libreta la documentación (o enlaces a dicha documentación) y un ejemplo de uso de una API pobremente documentada.

In [ ]:
# Otros endpoints de RNPDNO

#Estos son los endpoints que se obtienen al hacer una busqueda por por --TODOS-- los estados
'https://versionpublicarnpdno.segob.gob.mx/Sociodemografico/Totales'
'https://versionpublicarnpdno.segob.gob.mx/Sociodemografico/MapChartEntidades'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartSexoEstados'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/AreaChartSexoAnio'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartSexoAnioMeses'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/AreaChartSexoRango'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartSexoNacionalidad'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartFoliosIniciadosActualizadosFiscalias'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartFoliosIniciadosActualizadosComisiones'
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartFoliosPortal'

#Cuando se hace la busqueda de una entidad especifica, además de los anteriores, se usan
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartSexoMunicipio'

#Y el de colonia
'https://versionpublicarnpdno.segob.gob.mx/SocioDemografico/BarChartSexoColonia'

La API que usaré es `http://open-notify.org/Open-Notify-API/`
Esta API tiene dos endpoints, el primero da la geolocalización de la Estación espacial internacional, y la segunda da los nombres de las personas que estan en este momento en el espacio.
Se ejemplifica su uso.

In [45]:
url = 'http://api.open-notify.org/iss-now.json'

response = requests.get(url)

location = response.json()

location

{'message': 'success',
 'timestamp': 1693188089,
 'iss_position': {'longitude': '12.2362', 'latitude': '42.3851'}}

Se puede usar una biblioteca llamada folium para ir marcando en un capa las coordenadas, trazando la trayectoria de la estación espacial.

In [61]:
import folium
import time as t

In [87]:
def traza(time = 60):
    url = 'http://api.open-notify.org/iss-now.json'
    response = requests.get(url)
    location = response.json()

    m = folium.Map(location=[
        location['iss_position']['latitude'],
        location['iss_position']['longitude']], zoom_start=time/10)

    for _ in range(time):
        response = requests.get(url)
        location = response.json()
        folium.Marker([
            location['iss_position']['latitude'], 
            location['iss_position']['longitude']],
            icon=folium.Icon(icon='circle', color='red')).add_to(m)

        t.sleep(1)

    return m

In [88]:
traza()